# Monosemantic Feature Extraction: Two Key Experiments

This notebook walks through two experiments that shaped the monosemantic feature extraction algorithm:

1. **Switching from cosine similarity to dot product** for the neighbor graph (commit `6eb536c`)
2. **Post-hoc spectral gap filtering** as a replacement for the minimality filter in the overcomplete case

### Background

The algorithm extracts monosemantic features from neural network representations using synthetic data with known ground truth. The pipeline:

1. Generate a feature basis (orthogonal or ε-orthogonal) and sparse linear combinations as representations
2. Build a neighbor graph — threshold pairwise similarity to identify which reps share a feature
3. Find monosemantic targets — reps dominated by a single feature
4. For each target: compute the nullspace of non-neighbors, project neighbors onto it, SVD → dominant direction is the extracted feature

Both experiments address failures that arise when features are **overcomplete** ($n > d$), forcing $\varepsilon > 0$ inter-feature correlations.

In [ ]:
import torch
import torch.nn.functional as F
import sys
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

sys.path.insert(0, '..')

from src.config import SyntheticConfig, ExtractionConfig, compute_coef_min
from src.synthetic import generate_feature_basis, generate_representations
from src.extraction import (
    resolve_tau, compute_tau_bounds, compute_nullspace, extract_feature,
    extract_all_features, build_neighbor_matrix, find_monosemantic_targets
)
from src.metrics import evaluate, match_features

plt.rcParams['figure.figsize'] = (10, 4)
plt.rcParams['figure.dpi'] = 100
print(f"PyTorch {torch.__version__}")

---
# Experiment 1: Dot Product vs Cosine Similarity

## The Problem

The neighbor graph decides which representations "share a feature." Originally, we used **cosine similarity** with a threshold $\tau$:

$$\text{neighbors}(r_i, r_j) = \mathbb{1}\left[\frac{|r_i \cdot r_j|}{\|r_i\| \|r_j\|} \geq \tau\right]$$

This works fine when features are orthogonal ($\varepsilon = 0$). But when $\varepsilon > 0$, cosine similarity **normalizes away magnitude information** that's critical for distinguishing shared-feature signal from ε-interference noise.

## Why Magnitude Matters

Consider two representations that share feature $f_1$ with small coefficients $c_{\min}$:

$$r_a = c_{\min} f_1 + c_{\max} f_2 + c_{\max} f_3, \quad r_b = c_{\min} f_1 + c_{\max} f_4 + c_{\max} f_5$$

**Dot product:** $r_a \cdot r_b = c_{\min}^2 \underbrace{\langle f_1, f_1 \rangle}_{=1} + \sum_{\text{cross}} c_i c_j \underbrace{\langle f_i, f_j \rangle}_{\leq \varepsilon}$

The shared feature contributes $c_{\min}^2$. Cross-feature interference contributes at most $k^2 c_{\max}^2 \varepsilon$.

**Cosine similarity** divides by $\|r_a\| \|r_b\| \approx k \cdot c_{\max}^2$, shrinking both signal and noise by the same factor. But the noise floor *relative to the signal* stays the same — we've gained nothing from the normalization, and we've lost the ability to set an absolute threshold.

**Dot product** preserves the absolute scale: shared signal $\sim c_{\min}^2$, noise $\sim k^2 c_{\max}^2 \varepsilon$. These are on completely different scales when $\varepsilon$ is small.

## Setup: Overcomplete Features ($n > d$, $\varepsilon > 0$)

We need $\varepsilon > 0$ to see the difference, so we use an overcomplete basis.

In [ ]:
torch.manual_seed(42)

d, n_feat = 16, 20  # 20 features in 16 dimensions → overcomplete
basis = generate_feature_basis(d, n_feat)
eps = basis.achieved_epsilon

print(f"d={d}, n={n_feat}")
print(f"Achieved ε = {eps:.4f}")
print(f"Welch bound = {basis.welch_bound:.4f}")
print(f"Converged: {basis.converged}")

In [ ]:
# Generate representations with controlled coefficients
# coef_factor=0 → coef_min = floor = 0.5, coef_max = 1.0
# This gives us predictable bounds.
torch.manual_seed(42)

syn_config = SyntheticConfig(
    d=d, n=n_feat, k=2,
    num_representations=200,
    sparsity_mode='fixed',
    positive_only=True,
    coef_min_floor=0.5, coef_max=1.0, coef_factor=0.0,
)

reps, coeffs = generate_representations(basis.features, syn_config, epsilon=eps)

coef_min = compute_coef_min(syn_config, eps)
print(f"k={syn_config.k}, coef_min={coef_min:.2f}, coef_max={syn_config.coef_max:.2f}")
print(f"Representations: {reps.shape}")

## Computing Ground-Truth Sharing

Two representations share a feature iff their support sets overlap. Let's compute the actual dot products and cosine similarities for sharing vs non-sharing pairs.

In [ ]:
# Ground truth: which pairs share a feature?
support = (coeffs != 0).float()  # (N, n)
gt_sharing = (support @ support.T) > 0  # True if any feature in common

# Compute pairwise dot products and cosine similarities
dots = reps @ reps.T
norms = torch.norm(reps, dim=1, keepdim=True)
cossim = dots / (norms @ norms.T + 1e-8)

# Extract off-diagonal pairs
N = reps.shape[0]
mask = torch.triu(torch.ones(N, N, dtype=torch.bool), diagonal=1)

sharing = gt_sharing[mask].numpy()
dots_flat = torch.abs(dots[mask]).numpy()
cossim_flat = torch.abs(cossim[mask]).numpy()

print(f"Pairs: {sharing.sum()} sharing, {(~sharing.astype(bool)).sum()} non-sharing")

In [ ]:
# Side-by-side comparison: dot product vs cosine similarity
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Dot product
ax = axes[0]
ax.hist(dots_flat[sharing == True], bins=50, alpha=0.6, label='Sharing', color='steelblue', density=True)
ax.hist(dots_flat[sharing == False], bins=50, alpha=0.6, label='Non-sharing', color='salmon', density=True)
ax.set_xlabel('|dot product|')
ax.set_ylabel('Density')
ax.set_title('Dot product: sharing vs non-sharing')
ax.legend()

# Cosine similarity
ax = axes[1]
ax.hist(cossim_flat[sharing == True], bins=50, alpha=0.6, label='Sharing', color='steelblue', density=True)
ax.hist(cossim_flat[sharing == False], bins=50, alpha=0.6, label='Non-sharing', color='salmon', density=True)
ax.set_xlabel('|cosine similarity|')
ax.set_ylabel('Density')
ax.set_title('Cosine similarity: sharing vs non-sharing')
ax.legend()

plt.tight_layout()
plt.show()

# Separation gaps
dot_gap = dots_flat[sharing == True].min() - dots_flat[sharing == False].max()
cos_gap = cossim_flat[sharing == True].min() - cossim_flat[sharing == False].max()
print(f"Dot product separation gap:  {dot_gap:+.4f} {'✓ SEPARABLE' if dot_gap > 0 else '✗ overlap'}")
print(f"Cosine sim separation gap:   {cos_gap:+.4f} {'✓ SEPARABLE' if cos_gap > 0 else '✗ overlap'}")

## The Tau Bounds: Old vs New

The old cosine-similarity bounds divided by the maximum representation norm squared, collapsing the gap. The new dot-product bounds operate on the raw quantities:

| | Old (cosine sim) | New (dot product) |
|---|---|---|
| $\tau_{\text{upper}}$ (min sharing) | $c_{\min}^2 / (k \cdot c_{\max}^2)$ | $c_{\min}^2$ |
| $\tau_{\text{lower}}$ (max non-sharing) | $k \cdot \varepsilon$ | $k^2 \cdot c_{\max}^2 \cdot \varepsilon$ |
| Separable when | $c_{\min}^2 > k^2 c_{\max}^2 \varepsilon$ | same condition |

The separability *condition* is mathematically equivalent, but the cosine-sim bounds were **empirically wrong** — the normalization interacted poorly with the actual similarity distributions, making the predicted bounds unreliable.

In [ ]:
# Compute and compare bounds
k = syn_config.k
c_min = coef_min
c_max = syn_config.coef_max

# New dot product bounds (current code)
tau_upper_dot, tau_lower_dot = compute_tau_bounds(k, eps, c_min, c_max)

# Old cosine similarity bounds (before commit 6eb536c)
max_norm_sq = k * c_max**2
tau_upper_cos = c_min * c_min / max_norm_sq
tau_lower_cos = k * k * c_max * c_max * eps / max_norm_sq

print("DOT PRODUCT BOUNDS (current):")
print(f"  τ_upper = c_min² = {c_min}² = {tau_upper_dot:.4f}")
print(f"  τ_lower = k²·c_max²·ε = {k}²·{c_max}²·{eps:.4f} = {tau_lower_dot:.4f}")
print(f"  Gap: {tau_upper_dot - tau_lower_dot:+.4f}")
print(f"  Separable: {tau_lower_dot < tau_upper_dot}")

print(f"\nCOSINE SIMILARITY BOUNDS (old, before 6eb536c):")
print(f"  τ_upper = c_min²/(k·c_max²) = {c_min}²/({k}·{c_max}²) = {tau_upper_cos:.4f}")
print(f"  τ_lower = k·ε = {k}·{eps:.4f} = {tau_lower_cos:.4f}")
print(f"  Gap: {tau_upper_cos - tau_lower_cos:+.4f}")
print(f"  Separable: {tau_lower_cos < tau_upper_cos}")

In [ ]:
# Validate bounds against actual data
actual_min_sharing_dot = dots_flat[sharing == True].min()
actual_max_nonsharing_dot = dots_flat[sharing == False].max()
actual_min_sharing_cos = cossim_flat[sharing == True].min()
actual_max_nonsharing_cos = cossim_flat[sharing == False].max()

print("BOUNDS vs ACTUAL (dot product):")
print(f"  τ_upper = {tau_upper_dot:.4f}  ≤  actual min sharing = {actual_min_sharing_dot:.4f}?  {tau_upper_dot <= actual_min_sharing_dot + 1e-4}")
print(f"  τ_lower = {tau_lower_dot:.4f}  ≥  actual max non-sharing = {actual_max_nonsharing_dot:.4f}?  {tau_lower_dot >= actual_max_nonsharing_dot - 1e-4}")

print(f"\nBOUNDS vs ACTUAL (cosine similarity):")
print(f"  τ_upper = {tau_upper_cos:.4f}  ≤  actual min sharing = {actual_min_sharing_cos:.4f}?  {tau_upper_cos <= actual_min_sharing_cos + 1e-4}")
print(f"  τ_lower = {tau_lower_cos:.4f}  ≥  actual max non-sharing = {actual_max_nonsharing_cos:.4f}?  {tau_lower_cos >= actual_max_nonsharing_cos - 1e-4}")

## Concrete Example: A Small-Coefficient Monosemantic Rep

This is the scenario where cosine similarity breaks. A monosemantic representation with a small coefficient gets normalized to a unit vector, making the shared-feature signal indistinguishable from ε-noise.

In [ ]:
# Construct a concrete example by hand
torch.manual_seed(42)
features = basis.features  # (20, 16)

# Monosemantic rep: only feature 0, small coefficient
r_mono = 0.5 * features[0]  # small coef on one feature

# Sharing rep: features 0 and 3, with large coef on feature 3
r_share = 0.5 * features[0] + 1.0 * features[3]

# Non-sharing rep: features 5 and 7, large coefficients
r_nonshare = 1.0 * features[5] + 1.0 * features[7]

# Compute metrics
dot_share = torch.abs(r_mono @ r_share).item()
dot_nonshare = torch.abs(r_mono @ r_nonshare).item()

cos_share = torch.abs(F.cosine_similarity(r_mono.unsqueeze(0), r_share.unsqueeze(0))).item()
cos_nonshare = torch.abs(F.cosine_similarity(r_mono.unsqueeze(0), r_nonshare.unsqueeze(0))).item()

print(f"r_mono = 0.5·f₀                  (‖r‖ = {r_mono.norm():.3f})")
print(f"r_share = 0.5·f₀ + 1.0·f₃        (‖r‖ = {r_share.norm():.3f})")
print(f"r_nonshare = 1.0·f₅ + 1.0·f₇     (‖r‖ = {r_nonshare.norm():.3f})")
print(f"\nε = {eps:.4f} (max inter-feature correlation)")
print(f"")
print(f"                        Dot product    Cosine sim")
print(f"  r_mono · r_share:     {dot_share:>10.4f}     {cos_share:>10.4f}     (SHOULD be high — they share f₀)")
print(f"  r_mono · r_nonshare:  {dot_nonshare:>10.4f}     {cos_nonshare:>10.4f}     (SHOULD be low — no shared feature)")

print(f"\nDot product gap:  {dot_share - dot_nonshare:+.4f}")
print(f"Cosine sim gap:   {cos_share - cos_nonshare:+.4f}")

if dot_share > dot_nonshare and cos_share < cos_nonshare:
    print(f"\n⚠️  Cosine similarity REVERSED the order! Non-sharing looks more similar.")
    print(f"    Dot product gets it right.")
elif dot_share - dot_nonshare > cos_share - cos_nonshare:
    print(f"\n→ Dot product provides cleaner separation ({dot_share - dot_nonshare:.4f} vs {cos_share - cos_nonshare:.4f})")

In [ ]:
# Break down the dot product to show WHY
print("Decomposing r_mono · r_share:")
shared_signal = 0.5 * 0.5 * (features[0] @ features[0]).item()
cross_03 = 0.5 * 1.0 * (features[0] @ features[3]).item()
print(f"  Shared feature (f₀·f₀): 0.5 × 0.5 × ⟨f₀,f₀⟩ = {shared_signal:.4f}")
print(f"  Cross-term (f₀·f₃):     0.5 × 1.0 × ⟨f₀,f₃⟩ = {cross_03:.4f}  (ε-interference)")
print(f"  Total: {shared_signal + cross_03:.4f}")

print(f"\nDecomposing r_mono · r_nonshare:")
cross_05 = 0.5 * 1.0 * (features[0] @ features[5]).item()
cross_07 = 0.5 * 1.0 * (features[0] @ features[7]).item()
print(f"  Cross-term (f₀·f₅):     0.5 × 1.0 × ⟨f₀,f₅⟩ = {cross_05:.4f}  (ε-interference)")
print(f"  Cross-term (f₀·f₇):     0.5 × 1.0 × ⟨f₀,f₇⟩ = {cross_07:.4f}  (ε-interference)")
print(f"  Total: {cross_05 + cross_07:.4f}  (pure noise)")

print(f"\n→ With dot product, the shared signal ({shared_signal:.4f}) dominates.")
print(f"  Cosine similarity divides by ‖r_mono‖·‖r_share‖ = {r_mono.norm():.3f}·{r_share.norm():.3f} = {(r_mono.norm() * r_share.norm()):.3f}")
print(f"  This normalization amplifies the relative contribution of ε-noise.")

## End-to-End Validation

Run the full extraction pipeline with dot product (current code) and compare to what cosine similarity would give.

In [ ]:
# Dot product: use the current code directly
torch.manual_seed(42)
tau_dot = resolve_tau(ExtractionConfig(tau=None, tau_margin=0.5, epsilon=0.0), syn_config, epsilon=eps)

nbr_mat_dot = build_neighbor_matrix(reps, tau_dot)
targets_dot = find_monosemantic_targets(nbr_mat_dot)

extracted_dot = extract_all_features(
    reps, ExtractionConfig(tau=None, tau_margin=0.5, epsilon=0.0),
    syn_config, use_minimality_filter=True, basis_epsilon=eps
)

metrics_dot = evaluate(extracted_dot, basis.features, reps, coeffs, match_threshold=0.9)

print(f"DOT PRODUCT (current, τ={tau_dot:.4f}):")
print(f"  Targets found: {len(targets_dot)}")
print(f"  Features extracted: {extracted_dot.shape[0]}")
print(f"  Recovery rate: {metrics_dot.recovery_rate:.1%} ({int(metrics_dot.recovery_rate * n_feat)}/{n_feat})")
print(f"  Mean alignment: {metrics_dot.mean_alignment:.4f}")

In [ ]:
# Cosine similarity: simulate the OLD behavior
# The old code normalized rows then thresholded cosine sim
def build_neighbor_matrix_cosine(representations, tau):
    """OLD behavior: cosine similarity neighbor graph."""
    norms = torch.norm(representations, dim=1, keepdim=True)
    X_norm = representations / (norms + 1e-8)
    cossim = X_norm @ X_norm.T
    return torch.abs(cossim) >= tau

# Use the old tau bounds
tau_cos = tau_lower_cos + 0.5 * (tau_upper_cos - tau_lower_cos)

nbr_mat_cos = build_neighbor_matrix_cosine(reps, tau_cos)

# Check neighbor graph quality
off_diag = ~torch.eye(N, dtype=torch.bool)

for label, nbr_mat, tau_val in [('Dot product', nbr_mat_dot, tau_dot), ('Cosine sim (old)', nbr_mat_cos, tau_cos)]:
    pred = nbr_mat[off_diag]
    actual = gt_sharing[off_diag]
    tp = (pred & actual).sum().item()
    fp = (pred & ~actual).sum().item()
    fn = (~pred & actual).sum().item()
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0
    print(f"{label} (τ={tau_val:.4f}):  precision={prec:.3f}  recall={rec:.3f}  TP={tp}  FP={fp}  FN={fn}")

### Takeaway

Dot product preserves magnitude, giving clean separation between sharing and non-sharing pairs. Cosine similarity normalizes this away, causing overlap in the distributions and misclassification in the neighbor graph. The switch to dot product (commit `6eb536c`) was essential for the overcomplete case.

---
# Experiment 2: Post-Hoc Spectral Gap Filtering

## The Problem: Minimality Filter Fails at 0% Recall

The standard pipeline uses a **minimality filter**: select representations whose neighbor count is a local minimum (fewer neighbors than all their neighbors). The intuition is that monosemantic reps, dominated by one feature, should have small neighbor sets.

In the overcomplete case with Bernoulli-Gaussian sparsity, this intuition is **exactly backwards**:

- **Monosemantic reps** (k=1) align with a single feature that may appear in *many* other representations → **high** neighbor count
- **Multi-feature reps** with unusual feature combinations may share features with *few* others → **low** neighbor count

The minimality filter picks low-neighbor-count reps as targets — these are the multi-feature ones. **0% recall on truly monosemantic representations.**

In [ ]:
# Reproduce the failure: overcomplete + Bernoulli-Gaussian
torch.manual_seed(42)

syn_config_oc = SyntheticConfig(
    d=50, n=60,
    num_representations=500,
    sparsity_mode='bernoulli_gaussian',
    k=6,  # expected active features
    positive_only=True,
    coef_factor=10.0, coef_max=1.0, coef_min_floor=0.1,
)

basis_oc = generate_feature_basis(d=50, n=60)
eps_oc = basis_oc.achieved_epsilon
reps_oc, coeffs_oc = generate_representations(basis_oc.features, syn_config_oc, epsilon=eps_oc)

# Find truly monosemantic reps
nnz_oc = (coeffs_oc != 0).sum(dim=1)
true_mono_indices = set(torch.where(nnz_oc == 1)[0].tolist())

print(f"d={syn_config_oc.d}, n={syn_config_oc.n}, ε={eps_oc:.4f}")
print(f"Sparsity distribution (BG, expected k={syn_config_oc.k}):")
for k_val, count in sorted(Counter(nnz_oc.tolist()).items())[:10]:
    bar = '█' * (count // 5)
    print(f"  k={k_val:2d}: {count:3d} {bar}")
print(f"\nTruly monosemantic (k=1): {len(true_mono_indices)}")

In [ ]:
# Build neighbor graph and run minimality filter
# Use a manual tau since auto-tau can be unreliable with BG sparsity
tau_oc = resolve_tau(
    ExtractionConfig(tau=None, tau_margin=0.5, epsilon=0.0),
    syn_config_oc, epsilon=eps_oc
)
print(f"Auto-derived τ = {tau_oc:.4f}")

nbr_mat_oc = build_neighbor_matrix(reps_oc, tau_oc)
nbr_counts_oc = nbr_mat_oc.sum(dim=1)

# Minimality filter
targets_oc = find_monosemantic_targets(nbr_mat_oc)
target_set = set(targets_oc.tolist())

# How many truly monosemantic reps were selected?
true_pos = true_mono_indices & target_set
recall = len(true_pos) / len(true_mono_indices) if len(true_mono_indices) > 0 else 0

print(f"\nMinimality filter results:")
print(f"  Targets selected: {len(targets_oc)}")
print(f"  Truly monosemantic among targets: {len(true_pos)}")
print(f"  Recall on k=1 reps: {recall:.0%}")

target_sparsities = nnz_oc[targets_oc]
print(f"  Sparsity of selected targets: {Counter(target_sparsities.tolist())}")

In [ ]:
# WHY: monosemantic reps have MORE neighbors, not fewer
if len(true_mono_indices) > 0:
    mono_idx = torch.tensor(list(true_mono_indices))
    mono_nbr = nbr_counts_oc[mono_idx].float()
    multi_idx = torch.tensor([i for i in range(500) if i not in true_mono_indices])
    multi_nbr = nbr_counts_oc[multi_idx].float()

    print(f"Neighbor counts:")
    print(f"  Monosemantic (k=1):  mean={mono_nbr.mean():.1f}, median={mono_nbr.median():.1f}")
    print(f"  Multi-feature (k>1): mean={multi_nbr.mean():.1f}, median={multi_nbr.median():.1f}")
    print(f"")
    if mono_nbr.mean() > multi_nbr.mean():
        print(f"→ Monosemantic reps have MORE neighbors ({mono_nbr.mean():.0f} vs {multi_nbr.mean():.0f})!")
        print(f"  The minimality filter selects the opposite of what we want.")
    else:
        print(f"→ In this sample, mono reps have fewer neighbors — filter may work.")
else:
    print("No k=1 reps in this sample (BG sparsity is stochastic).")
    # Show neighbor count distribution by sparsity level
    print(f"\nNeighbor counts by sparsity:")
    for k_val in sorted(set(nnz_oc.tolist()))[:8]:
        k_mask = (nnz_oc == k_val)
        if k_mask.sum() > 0:
            k_nbr = nbr_counts_oc[k_mask].float()
            print(f"  k={k_val}: mean={k_nbr.mean():.1f} neighbors ({k_mask.sum()} reps)")

In [ ]:
# Visualize: neighbor counts by sparsity level
fig, ax = plt.subplots(figsize=(10, 5))

if len(true_mono_indices) > 0:
    ax.hist(mono_nbr.numpy(), bins=30, alpha=0.6, label='k=1 (monosemantic)', density=True, color='steelblue')
    ax.hist(multi_nbr.numpy(), bins=30, alpha=0.5, label='k>1 (multi-feature)', density=True, color='salmon')
else:
    # Color by low-k vs high-k
    low_k = (nnz_oc <= 3)
    high_k = (nnz_oc > 6)
    if low_k.sum() > 0:
        ax.hist(nbr_counts_oc[low_k].float().numpy(), bins=30, alpha=0.6, label='k≤3 (sparse)', density=True, color='steelblue')
    if high_k.sum() > 0:
        ax.hist(nbr_counts_oc[high_k].float().numpy(), bins=30, alpha=0.5, label='k>6 (dense)', density=True, color='salmon')

ax.set_xlabel('Neighbor count')
ax.set_ylabel('Density')
ax.set_title('Neighbor count distribution by true sparsity')
ax.legend()
plt.tight_layout()
plt.show()

## The Alternative: Extract Everything, Filter by Spectral Gap

Since we can't identify monosemantic targets *before* extraction, we flip the approach:

1. **Extract from ALL unique neighbor-set representatives** (disable minimality filter)
2. For each extraction, record the singular value spectrum from the nullspace projection
3. **Use $\sigma_1 / \sigma_2$ (the spectral gap) to identify monosemantic extractions post-hoc**

The intuition: when a target's neighbors truly share one dominant feature, projecting them onto the nullspace of non-neighbors yields a rank-1 matrix — one huge singular value, everything else negligible. A large spectral gap $\sigma_1 / \sigma_2 \gg 1$ is the signature of monosemanticity.

Multi-feature targets will have more evenly distributed singular values (multiple significant directions in the projection).

In [ ]:
# Extract from ALL unique neighbor-set representatives, recording spectral gaps
from src.extraction import cluster_by_neighbors

clusters_oc = cluster_by_neighbors(reps_oc, tau_oc)
print(f"Unique neighbor sets: {len(clusters_oc)}")

spectral_gaps = []
all_extracted = []
all_target_idx = []
all_singular_values = []
failed = 0

for neighbor_set, member_indices in clusters_oc.items():
    target_idx = member_indices[0]
    nbr_indices = torch.tensor(sorted(neighbor_set), dtype=torch.long)

    if len(nbr_indices) < 2:
        failed += 1
        continue

    try:
        nullspace = compute_nullspace(reps_oc, nbr_indices, epsilon=0.0)
        if nullspace.shape[0] == 0:
            failed += 1
            continue

        neighbors = reps_oc[nbr_indices]
        projected = neighbors @ nullspace.T @ nullspace
        _, S, Vh = torch.linalg.svd(projected, full_matrices=False)

        feature = Vh[0]
        gap = (S[0] / S[1]).item() if len(S) > 1 and S[1] > 1e-10 else float('inf')

        all_extracted.append(feature)
        spectral_gaps.append(gap)
        all_target_idx.append(target_idx)
        all_singular_values.append(S[:5].tolist())
    except (ValueError, RuntimeError):
        failed += 1
        continue

all_extracted_t = torch.stack(all_extracted) if all_extracted else torch.empty(0, 50)
spectral_gaps_t = torch.tensor(spectral_gaps)

print(f"Successful extractions: {len(all_extracted)}")
print(f"Failed: {failed}")
print(f"Spectral gap — min: {spectral_gaps_t.min():.2f}, median: {spectral_gaps_t.median():.2f}, max: {spectral_gaps_t.max():.2f}")

In [ ]:
# Match each extraction to ground truth
sims_to_gt = torch.abs(all_extracted_t @ basis_oc.features.T)  # (m, n)
best_sim_per = sims_to_gt.max(dim=1).values
is_correct = best_sim_per > 0.9

print(f"Extractions matching a GT feature (|cos_sim| > 0.9): {is_correct.sum().item()} / {len(all_extracted)}")

correct_gaps = spectral_gaps_t[is_correct]
incorrect_gaps = spectral_gaps_t[~is_correct]

print(f"\nSpectral gaps — correct:   median={correct_gaps.median():.2f}, mean={correct_gaps.mean():.2f}" if is_correct.sum() > 0 else "No correct extractions")
print(f"Spectral gaps — incorrect: median={incorrect_gaps.median():.2f}, mean={incorrect_gaps.mean():.2f}" if (~is_correct).sum() > 0 else "All correct!")

In [ ]:
# Spectral gap histogram: correct vs incorrect
fig, ax = plt.subplots(figsize=(10, 5))

max_display = min(spectral_gaps_t.quantile(0.98).item() * 1.5, 100)

if is_correct.sum() > 0:
    ax.hist(correct_gaps.clamp(max=max_display).numpy(), bins=40, alpha=0.6,
            label=f'Correct ({is_correct.sum().item()})', color='steelblue')
if (~is_correct).sum() > 0:
    ax.hist(incorrect_gaps.clamp(max=max_display).numpy(), bins=40, alpha=0.6,
            label=f'Incorrect ({(~is_correct).sum().item()})', color='salmon')
ax.set_xlabel('Spectral gap (σ₁/σ₂)')
ax.set_ylabel('Count')
ax.set_title('Spectral gap distribution: correct vs incorrect extractions')
ax.legend()
plt.tight_layout()
plt.show()

## Precision/Recall at Different Spectral Gap Thresholds

We can sweep the spectral gap threshold to trade off precision (fraction of selected features that are correct) vs recall (fraction of ground-truth features recovered).

In [ ]:
from src.extraction import _deduplicate_features

thresholds = [1.0, 1.5, 2.0, 3.0, 5.0, 8.0, 10.0, 15.0, 20.0, 30.0, 50.0]
n_gt = syn_config_oc.n

print(f"{'Threshold':>10} {'Selected':>8} {'Deduped':>7} {'Matched':>7} {'Precision':>9} {'Recall':>7}")
print("-" * 55)

pr_precisions = []
pr_recalls = []

for thresh in thresholds:
    sel = spectral_gaps_t >= thresh
    if sel.sum() == 0:
        print(f"{thresh:>10.1f} {'—':>8}")
        pr_precisions.append(1.0)
        pr_recalls.append(0.0)
        continue

    sel_features = all_extracted_t[sel]
    if sel_features.shape[0] > 1:
        sel_features = _deduplicate_features(sel_features, threshold=0.95)

    matching, scores = match_features(sel_features, basis_oc.features, threshold=0.9)
    prec = len(matching) / sel_features.shape[0] if sel_features.shape[0] > 0 else 0
    rec = len(matching) / n_gt

    pr_precisions.append(prec)
    pr_recalls.append(rec)

    print(f"{thresh:>10.1f} {sel.sum().item():>8} {sel_features.shape[0]:>7} {len(matching):>7} {prec:>8.1%} {rec:>7.1%}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Precision and recall vs threshold
ax = axes[0]
ax.plot(thresholds, pr_precisions, 'b-o', markersize=5, linewidth=2, label='Precision')
ax.plot(thresholds, pr_recalls, 'r-s', markersize=5, linewidth=2, label='Recall')
ax.set_xlabel('Spectral gap threshold (σ₁/σ₂)')
ax.set_ylabel('Rate')
ax.set_title('Precision/Recall vs spectral gap threshold')
ax.set_xscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

# PR curve
ax = axes[1]
ax.plot(pr_recalls, pr_precisions, 'g-o', markersize=5, linewidth=2)
for i, t in enumerate(thresholds):
    if i % 2 == 0 and pr_recalls[i] > 0:
        ax.annotate(f'{t:.0f}', (pr_recalls[i], pr_precisions[i]),
                    textcoords='offset points', xytext=(5, 5), fontsize=8)
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('Precision-Recall curve')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Multi-Feature Targets Can Yield Correct Features

An interesting finding: you don't *need* a truly monosemantic target (k=1) to extract a correct feature. If one feature in a multi-feature target dominates the neighbor set's shared direction, the SVD still picks it out. The spectral gap captures this.

In [ ]:
# For each extraction, check the true sparsity of the target rep
target_true_k = nnz_oc[torch.tensor(all_target_idx)]

print(f"{'k':>4} | {'Total':>6} | {'Correct':>7} | {'Rate':>6} | {'Median gap':>10}")
print("-" * 50)

for k_val in sorted(set(target_true_k.tolist())):
    mask_k = (target_true_k == k_val)
    total = mask_k.sum().item()
    correct_k = (is_correct & mask_k).sum().item()
    gaps_k = spectral_gaps_t[mask_k]
    med_gap = gaps_k.median().item() if len(gaps_k) > 0 else 0
    print(f"{k_val:>4} | {total:>6} | {correct_k:>7} | {correct_k/total if total > 0 else 0:>5.1%} | {med_gap:>10.2f}")

In [ ]:
# Show a concrete example: singular value spectrum from a good vs bad extraction
# Find the extraction with the highest spectral gap that's correct
if is_correct.sum() > 0:
    correct_indices = torch.where(is_correct)[0]
    best_correct = correct_indices[spectral_gaps_t[correct_indices].argmax()].item()

    # Find a bad extraction (low gap, incorrect)
    if (~is_correct).sum() > 0:
        incorrect_indices = torch.where(~is_correct)[0]
        worst_incorrect = incorrect_indices[spectral_gaps_t[incorrect_indices].argmin()].item()
    else:
        worst_incorrect = None

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # Good extraction
    sv_good = all_singular_values[best_correct]
    ax = axes[0]
    ax.bar(range(1, len(sv_good)+1), sv_good, color='steelblue', edgecolor='black')
    ax.set_xlabel('Singular value index')
    ax.set_ylabel('σ')
    k_good = nnz_oc[all_target_idx[best_correct]].item()
    ax.set_title(f'Correct extraction (gap={spectral_gaps[best_correct]:.1f}, target k={k_good})')

    # Bad extraction
    if worst_incorrect is not None:
        sv_bad = all_singular_values[worst_incorrect]
        ax = axes[1]
        ax.bar(range(1, len(sv_bad)+1), sv_bad, color='salmon', edgecolor='black')
        ax.set_xlabel('Singular value index')
        ax.set_ylabel('σ')
        k_bad = nnz_oc[all_target_idx[worst_incorrect]].item()
        ax.set_title(f'Incorrect extraction (gap={spectral_gaps[worst_incorrect]:.1f}, target k={k_bad})')

    plt.tight_layout()
    plt.show()
    
    print(f"Left: one dominant singular value → monosemantic direction found")
    print(f"Right: multiple significant singular values → no single dominant feature")

---
# Summary

## Experiment 1: Cosine Similarity → Dot Product

**Problem:** Cosine similarity normalizes by $\|r_i\|\|r_j\|$, destroying magnitude information. When $\varepsilon > 0$, small-coefficient shared features become indistinguishable from cross-feature interference after normalization.

**Fix:** Switch to raw dot product. The new $\tau$ bounds are:
$$\tau_{\text{upper}} = c_{\min}^2 \quad \text{(min sharing signal)}$$
$$\tau_{\text{lower}} = k^2 c_{\max}^2 \varepsilon \quad \text{(max non-sharing noise)}$$

Separable when $c_{\min}^2 > k^2 c_{\max}^2 \varepsilon$. The bounds directly reflect the physics of the data generation.

## Experiment 2: Post-Hoc Spectral Gap Filtering

**Problem:** The minimality filter (select targets with fewest neighbors) has 0% recall in the overcomplete case. Monosemantic reps have *high* neighbor counts because their single feature is shared widely. Multi-feature reps with unusual combinations have *low* counts.

**Fix:** Extract from *all* unique neighbor-set representatives, then filter post-hoc using the spectral gap $\sigma_1 / \sigma_2$ from the nullspace-projected SVD. A large gap indicates a single dominant feature direction was found — the hallmark of a monosemantic extraction.

**Key finding:** Even multi-feature targets can yield correct features if one feature dominates the neighbor set's shared direction. The spectral gap captures this automatically.